In [1]:
#pip install sentence-transformers

In [3]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
#import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import networkx as nx
import json
import langid
from sklearn.feature_extraction.text import TfidfVectorizer
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk import pos_tag, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from collections import Counter
import ast

# Download the NLTK data for part-of-speech tagging
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vandi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vandi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# Creating the Dataset

## Creating CSV from JSON

In [6]:
json_file=pd.read_json('sm_pads.json', orient='records')

In [7]:
pads_list = json_file['pads']
flattened_data = []

for pad in pads_list:
    flattened_pad = {
        'id': pad['id'],
        'content': pad['content'],
        'is_public': pad['is_public'],
        'tags': '; '.join(pad['tags']),  # Join the list of tags into a string
        'sdgs': '; '.join(pad['sdgs']),  # Join the list of sdgs into a string
        'country': pad['country'],
        'lang': pad['lang']
    }
    flattened_data.append(flattened_pad)

In [8]:
df = pd.DataFrame(flattened_data)
df.head()

,id,content,is_public,tags,sdgs,country,lang
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,clean energy; affordable energy; stove,Good health and well-being; Affordable and cle...,ETH,en
1,4571,Alokito hridoy - enlightening the souls of chi...,True,education,Quality education,BGD,en
2,6099,BioFabricating Materials\nSolution holder: Bat...,False,nature-based solutions; design; sustainable fa...,"Industry, innovation and infrastructure; Respo...",JOR,en
3,6100,Circulamic\nSolution holder: Batoul Al-Rashdan...,False,nature-based solutions; recycling; design; sus...,"Industry, innovation and infrastructure; Respo...",JOR,en
4,4145,Gakyid Ride (City Bus ride app)\nThis app prov...,False,digital platform; innovation challenges; innov...,Good health and well-being; Sustainable cities...,BTN,en


In [9]:
df.dropna(inplace=True)
df.shape
df['tags'] = df['tags'].str.split('; ')
df_explode=df.explode('tags')
tag_distribution = df_explode['tags'].value_counts()
#df['tags'] = df['tags'].apply(ast.literal_eval)

In [10]:
#df_explode = df.explode('tags')

In [11]:
print(df['tags'][0])

['clean energy', 'affordable energy', 'stove']


In [12]:
df_explode.head()

,id,content,is_public,tags,sdgs,country,lang
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,clean energy,Good health and well-being; Affordable and cle...,ETH,en
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,affordable energy,Good health and well-being; Affordable and cle...,ETH,en
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,stove,Good health and well-being; Affordable and cle...,ETH,en
1,4571,Alokito hridoy - enlightening the souls of chi...,True,education,Quality education,BGD,en
2,6099,BioFabricating Materials\nSolution holder: Bat...,False,nature-based solutions,"Industry, innovation and infrastructure; Respo...",JOR,en


In [13]:
df_explode['unique_id'] = range(len(df_explode))
df_explode = df_explode[['unique_id'] + [col for col in df_explode.columns if col != 'unique_id']]
print(df_explode[df_explode['tags'] == 'clean energy'].iloc[0])

unique_id                                                    0
id                                                        4364
content      Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...
is_public                                                 True
tags                                              clean energy
sdgs         Good health and well-being; Affordable and cle...
country                                                    ETH
lang                                                        en
Name: 0, dtype: object


In [14]:
df_explode.head()

,unique_id,id,content,is_public,tags,sdgs,country,lang
0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,clean energy,Good health and well-being; Affordable and cle...,ETH,en
0,1,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,affordable energy,Good health and well-being; Affordable and cle...,ETH,en
0,2,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,stove,Good health and well-being; Affordable and cle...,ETH,en
1,3,4571,Alokito hridoy - enlightening the souls of chi...,True,education,Quality education,BGD,en
2,4,6099,BioFabricating Materials\nSolution holder: Bat...,False,nature-based solutions,"Industry, innovation and infrastructure; Respo...",JOR,en


In [15]:
df_explode.set_index('unique_id', inplace=True)

In [16]:
df_explode = df_explode.apply(map(lambda s: s.lower() if type(s) == str else s))

TypeError: map() must have at least two arguments.

In [18]:
df.to_csv("capstone_data_final.csv")

## Extract Keywords

In [19]:
def remove_non_english_tokens(text):
    '''Remove all non english tokens'''
    # Split the text into tokens

    # Identify the language of each token
    english_tokens = [word for word in text.split() if langid.classify(word)[0] == 'en']

    # Reconstruct the text with English tokens
    filtered_text = ' '.join(english_tokens)

    return filtered_text

In [20]:
def remove_proper_nouns(text):
    '''remove proper nouns such as names of persons'''
    # Tokenize the text into words
    words = word_tokenize(text)

    # Perform part-of-speech tagging
    pos_tags = pos_tag(words)

    # Remove proper nouns (NNP: singular proper noun, NNPS: plural proper noun)
    filtered_words = [word for word, pos in pos_tags if pos not in ['NNP', 'NNPS']]

    # Reconstruct the text without proper nouns
    filtered_text = ' '.join(filtered_words)

    return filtered_text

In [21]:
def remove_emails_and_hyperlinks(text):
    '''remove emails and hyperlinks'''
    # Remove emails
    text_no_emails = re.sub(r'\S+@\S+', '', text)

    # Remove hyperlinks
    text_no_links = re.sub(r'http[s]?\S+', '', text_no_emails,flags=re.IGNORECASE)

    return text_no_links

In [22]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [23]:
df=pd.read_csv("capstone_data_final.csv")
df=df[df['lang']=='en']

In [24]:
df["clean_content"]=df["content"].apply(remove_non_english_tokens)
df["clean_content"]=df["clean_content"].apply(remove_emails_and_hyperlinks)
df["clean_content"]=df["clean_content"].apply(remove_proper_nouns)
df["clean_content"]=df["clean_content"].apply(remove_numbers)

In [25]:
model = KeyBERT(model="distilbert-base-nli-mean-tokens")

# Function to extract keywords (tags) for each text
def extract_tags(text, top_n=5,keyprob=False):
    '''extract top keywords based on the parameter given by user'''
    keywords = model.extract_keywords(text,top_n=top_n)
    return [keyword[0] for keyword in keywords if keyword[1]>=0.1]

# Apply the extract_tags function to each text in the DataFrame
df["tags_keybert"] = df["clean_content"].apply(lambda text: extract_tags(text, top_n=20,keyprob=True))

In [26]:
df.to_csv("keybert_thresh_onlyen_0.1_nltk.csv")

# Level-1: Semantic clustering of the tags (user assigned + KeyBERT) using SBERT

In [27]:
combined_df = pd.read_csv("keybert_thresh_onlyen_0.1_nltk.csv")

In [28]:
combined_df['tags'] = combined_df['tags'].apply(ast.literal_eval)
combined_df['tags_keybert'] = combined_df['tags_keybert'].apply(ast.literal_eval)

In [29]:
combined_df['combined_tags'] = combined_df['tags'] + combined_df['tags_keybert']

In [30]:
combined_df

,Unnamed: 0.1,Unnamed: 0,id,content,is_public,tags,sdgs,country,lang,clean_content,tags_keybert,combined_tags
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...","[clean energy, affordable energy, stove, stove..."
1,1,1,4571,Alokito hridoy - enlightening the souls of chi...,True,[education],Quality education,BGD,en,hridoy - the souls of children ( meaning soul ...,"[teachers, teacher, training, education, learn...","[education, teachers, teacher, training, educa..."
2,2,2,6099,BioFabricating Materials\nSolution holder: Bat...,False,"[nature-based solutions, design, sustainable f...","Industry, innovation and infrastructure; Respo...",JOR,en,BioFabricating Materials holder : In this last...,"[biofabricating, plastics, crafting, technolog...","[nature-based solutions, design, sustainable f..."
3,3,3,6100,Circulamic\nSolution holder: Batoul Al-Rashdan...,False,"[nature-based solutions, recycling, design, su...","Industry, innovation and infrastructure; Respo...",JOR,en,Circulamic holder : by is a clever way to add ...,"[customizable, tools, acoustics, lab, helping,...","[nature-based solutions, recycling, design, su..."
4,4,4,4145,Gakyid Ride (City Bus ride app)\nThis app prov...,False,"[digital platform, innovation challenges, inno...",Good health and well-being; Sustainable cities...,BTN,en,( ride app ) This app provides commuters with ...,"[bus, buses, traffic, commuters]","[digital platform, innovation challenges, inno..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2774,3382,3382,6083,Container Filter\n\nFilter to capture cediment...,False,"[climate change, clean water]",Clean water and sanitation; Responsible consum...,GTM,en,to capture in the water stored in the rotopla ...,"[tanks, water, stored, rotopla, capture]","[climate change, clean water, tanks, water, st..."
2775,3383,3383,6084,Galvanized gutter\n\nGalvanized gutter for wat...,False,"[climate change, clean water]",Clean water and sanitation; Responsible consum...,GTM,en,for water transport .,"[water, transport]","[climate change, clean water, water, transport]"
2776,3384,3384,6085,Iron Gutter\n\nGalvanized gutter for water tra...,False,"[climate change, clean water, rain water colle...",Clean water and sanitation; Responsible consum...,GTM,en,for water transport,"[water, transport]","[climate change, clean water, rain water colle..."
2777,3386,3386,6089,PVC pipe for rain descent\n\nPVC pipe for rain...,False,"[climate change, clean water, water mobilization]",Clean water and sanitation; Responsible consum...,GTM,en,pipe for rain descent pipe for rain descent . ...,"[pipe, rain, cross, descent, section, used]","[climate change, clean water, water mobilizati..."


In [31]:
combined_df_explode = combined_df.explode('combined_tags')

In [32]:
combined_df_explode

,Unnamed: 0.1,Unnamed: 0,id,content,is_public,tags,sdgs,country,lang,clean_content,tags_keybert,combined_tags
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...",clean energy
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...",affordable energy
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...",stove
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...",stove
0,0,0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,"[clean energy, affordable energy, stove]",Good health and well-being; Affordable and cle...,ETH,en,"stove contact address , -- -- stove is used f...","[stove, firewood, cooking, combustion, clay, g...",firewood
...,...,...,...,...,...,...,...,...,...,...,...,...
2778,3387,3387,6091,T Connection\n\nPVC T for connection between d...,False,"[clean energy, climate change, water mobilizat...",Clean water and sanitation; Responsible consum...,GTM,en,for connection between drainage pipes .,"[drainage, pipes, connection]",climate change
2778,3387,3387,6091,T Connection\n\nPVC T for connection between d...,False,"[clean energy, climate change, water mobilizat...",Clean water and sanitation; Responsible consum...,GTM,en,for connection between drainage pipes .,"[drainage, pipes, connection]",water mobilization
2778,3387,3387,6091,T Connection\n\nPVC T for connection between d...,False,"[clean energy, climate change, water mobilizat...",Clean water and sanitation; Responsible consum...,GTM,en,for connection between drainage pipes .,"[drainage, pipes, connection]",drainage
2778,3387,3387,6091,T Connection\n\nPVC T for connection between d...,False,"[clean energy, climate change, water mobilizat...",Clean water and sanitation; Responsible consum...,GTM,en,for connection between drainage pipes .,"[drainage, pipes, connection]",pipes


In [33]:
print(combined_df_explode['combined_tags'][0])

0         clean energy
0    affordable energy
0                stove
0                stove
0             firewood
0              cooking
0           combustion
0                 clay
0           galvanized
0              chamber
0                metal
0               market
0             internal
0             diameter
0                 high
0                  fed
0              opening
0                inlet
0                 open
0                inner
0                liner
0                sheet
0                scale
Name: combined_tags, dtype: object


In [34]:
combined_df_explode['unique_id'] = range(len(combined_df_explode))
combined_df_explode = combined_df_explode[['unique_id'] + [col for col in combined_df_explode.columns if col != 'unique_id']]
print(combined_df_explode[combined_df_explode['combined_tags'] == 'clean energy'].iloc[0])



unique_id                                                        0
Unnamed: 0.1                                                     0
Unnamed: 0                                                       0
id                                                            4364
content          Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...
is_public                                                     True
tags                      [clean energy, affordable energy, stove]
sdgs             Good health and well-being; Affordable and cle...
country                                                        ETH
lang                                                            en
clean_content    stove contact address ,  -- -- stove is used f...
tags_keybert     [stove, firewood, cooking, combustion, clay, g...
combined_tags                                         clean energy
Name: 0, dtype: object


In [35]:
combined_df_explode.set_index('unique_id', inplace=True)

In [36]:
combined_df_explode = combined_df_explode.applymap(lambda s: s.lower() if type(s) == str else s)

In [37]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [38]:
combined_corpus = combined_df_explode['combined_tags'].unique()

In [39]:
combined_corpus = list(combined_corpus)

In [40]:
combined_corpus

['clean energy',
 'affordable energy',
 'stove',
 'firewood',
 'cooking',
 'combustion',
 'clay',
 'galvanized',
 'chamber',
 'metal',
 'market',
 'internal',
 'diameter',
 'high',
 'fed',
 'opening',
 'inlet',
 'open',
 'inner',
 'liner',
 'sheet',
 'scale',
 'education',
 'teachers',
 'teacher',
 'training',
 'learning',
 'students',
 'educating',
 'workers',
 'factory',
 'nature-based solutions',
 'design',
 'sustainable fashion',
 'biofabricating',
 'plastics',
 'crafting',
 'technologies',
 'craftsmanship',
 'century',
 'craft',
 'research',
 'recycling',
 'customizable',
 'tools',
 'acoustics',
 'lab',
 'helping',
 'useful',
 'helps',
 'modular',
 'flexibility',
 'disguise',
 'recycled',
 'digital platform',
 'innovation challenges',
 'innovation during covid-19',
 'reduction of emissions',
 'bus',
 'buses',
 'traffic',
 'commuters',
 'water supply',
 'wind pump',
 'rural',
 'pump',
 'pumping',
 'wind',
 'water',
 'powered',
 'incubation',
 'hammer',
 'drive',
 'acting',
 'energy

In [41]:
# corpus_embeddings = embedder.encode(corpus)
combined_corpus_embeddings = embedder.encode(combined_corpus, show_progress_bar=True)

Batches:   0%|          | 0/188 [00:00<?, ?it/s]

In [42]:
combined_corpus_embeddings

array([[-0.05087513,  0.11080185,  0.07873712, ..., -0.03314728,
        -0.00361693, -0.04562355],
       [ 0.00943787,  0.0832081 ,  0.02939451, ..., -0.08229157,
        -0.02408933,  0.02009457],
       [-0.04290591,  0.06958827, -0.06878943, ...,  0.04267294,
        -0.01066506, -0.01831763],
       ...,
       [-0.01114256, -0.00912488, -0.06040759, ..., -0.06915261,
         0.04999437,  0.06525677],
       [ 0.03783558,  0.05597052,  0.0066634 , ...,  0.00047348,
        -0.01040438, -0.03141515],
       [-0.02131408,  0.02359336, -0.01075834, ...,  0.04949199,
         0.01815608, -0.00782548]], dtype=float32)

In [43]:
combined_corpus_embeddings = combined_corpus_embeddings /  np.linalg.norm(combined_corpus_embeddings, axis=1, keepdims=True)

In [71]:
# clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
combined_clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold = 0.78, metric='cosine', linkage='average')
combined_clustering_model.fit(combined_corpus_embeddings)
combined_cluster_assignment = combined_clustering_model.labels_

In [72]:
combined_clustered_sentences = {}
for sentence_id, cluster_id in enumerate(combined_cluster_assignment):
    if cluster_id not in combined_clustered_sentences:
        combined_clustered_sentences[cluster_id] = []

    combined_clustered_sentences[cluster_id].append(combined_corpus[sentence_id])

In [73]:
combined_sorted_clusters = sorted(combined_clustered_sentences.items(), key=lambda x: x[0])
for i, (cluster_id, cluster) in enumerate(combined_sorted_clusters):
    print("Cluster: ", i+1)
    print(cluster)
    print("")

Cluster:  1
['twinges', 'yucca', 'rawabi', 'ccoking', 'batrina', 'khattab', 'revadex', 'pinpep', 'probosque', 'circassian', 'gubha', 'boost', 'olive', 'taro', 'jewlery', 'matrerial', 'mestizo', 'giza', 'usturlab', 'tilapia', 'tarpaulins', 'lok', 'boosters', 'booster', 'harrow', 'speedgel', 'chorizo', 'cocoa', 'aprocane', 'tango', 'cero', 'ccookstoves', 'ikalany', 'wenda', 'vodafone', 'leapfrog', 'pallet', 'racemosa', 'sargassum', 'helical', 'baobab', 'coronapp', 'rocket', 'mats', 'shingles', 'thistleandhoneyy', 'bicicleta', 'jalisco', 'pallets', 'batik', 'paulina', 'bonivente', 'cristian', 'fernanda', 'medina', 'sars', 'ttc', 'oxen', 'cococnut', 'palsy', 'trycicle', 'ingrid', 'mobi', 'noblex', 'letsfarmapp', 'masika', 'bithum', 'emmanuel', 'twine', 'matooke', 'empire', 'coco', 'amaranthus', 'kulitis', 'tarred', 'nelsonioides', 'sargablocks', 'foster', 'copra', 'vetiver', 'olives', 'fern', 'doubles', 'dalo', 'cress', 'rod', 'rockethealth', 'palm', 'artesano', 'proyecto', 'proyectovaquit

In [47]:
combined_cluster_assignment

array([75, 75, 72, ..., 21, 66, 66], dtype=int64)

In [48]:
combined_clustering_model.get_params()

{'affinity': 'deprecated',
 'compute_distances': False,
 'compute_full_tree': 'auto',
 'connectivity': None,
 'distance_threshold': 0.73,
 'linkage': 'average',
 'memory': None,
 'metric': 'cosine',
 'n_clusters': None}

In [53]:
type(combined_clustered_sentences)

dict

In [74]:
sbert_clusters = {key: [value.strip("'") for value in values] for key, values in combined_clustered_sentences.items()}

In [75]:
len(sbert_clusters.keys())

45

# Reading in the clusters and applying GloVe to get the root word

In [56]:
cluster_info_file_path = 'level-1-final-sbert-refined_new_0.73.txt'   #93 clusters
cluster_info = {}

with open(cluster_info_file_path, 'r') as file:
    lines = file.readlines()
    
#     for line in file:
#         cluster_name=line.strip().split('\t')
#         print(cluster_name[0].split()[0])
#         tag, cluster_id =cluster_name[0].split()
#         cluster_info[tag] = int(cluster_id)
        

# Initialize an empty dictionary to store clusters
clusters_dict = {}

# Process each line in the file
current_cluster = None
for line in lines:
    # Strip leading and trailing whitespaces
    line = line.strip()

    # Check if the line starts with 'Cluster'
    if line.startswith('Cluster'):
        current_cluster = line  # Set the current cluster name
        clusters_dict[current_cluster] = []  # Initialize an empty list for the cluster
    elif line.startswith('[') and line.endswith(']'):
        # If inside a cluster and the line is a list, convert it to a list and append to the dictionary
        clusters_dict[current_cluster] = line

# Convert string representations to actual lists using a custom function
def convert_to_list(s):
    try:
        # Assuming the string is enclosed in square brackets, remove extra whitespaces and evaluate as a list
        return [item.strip() for item in s[1:-1].split(',')]
    except (ValueError, TypeError, IndexError):
        # Handle errors gracefully
        return []

converted_clusters = {key: convert_to_list(value) for key, value in clusters_dict.items()}
sbert_clusters = {key: [value.strip("'") for value in values] for key, values in converted_clusters.items()}

In [ ]:
# #clusters_dict
# total=0
# for idx, tlist in sbert_clusters.items():
#     #print(len(tlist))
#     total+=len(tlist)
# print(total)

In [57]:
sbert_clusters

{'Cluster  1': ['income generation',
  'youth and unemployment',
  'employment',
  'holder',
  'grocery',
  'product',
  'self-employment',
  'innovative goods',
  'business',
  'shopping',
  'market',
  'saving cost',
  'marketing',
  'unemployment',
  'industry',
  'creative economy',
  'alternative finance',
  'logistics',
  'price',
  'factory',
  'resource management',
  'sale',
  'ecommerce',
  'goods',
  'leadership',
  'store',
  'work',
  'management',
  'buyer',
  'quality',
  'author',
  'wealth',
  'shop',
  'economy',
  'commerce',
  'market need',
  'owner',
  'street vendors',
  'job',
  'doctor',
  'staff',
  'selling',
  'executive',
  'fund',
  'discount',
  'coupon',
  'blue economy',
  'consumer',
  'income',
  'tradeunions',
  'trade',
  'demand',
  'supply',
  'expense',
  'resource',
  'retail',
  'warehouse',
  'fee',
  'e-commerce',
  'bank',
  'company',
  'banking',
  'finance',
  'jobs',
  'insider',
  'labour',
  'human resources',
  'recruitment',
  'depar

## GloVe word embeddings and word generation

## Final 

In [ ]:
cluster_info_file_path = 'glove_0.78.txt' # File name- file containing the semantic clusters
cluster_info = {}

with open(cluster_info_file_path, 'r') as file:
    lines = file.readlines()
    
#     for line in file:
#         cluster_name=line.strip().split('\t')
#         print(cluster_name[0].split()[0])
#         tag, cluster_id =cluster_name[0].split()
#         cluster_info[tag] = int(cluster_id)
        

# Initialize an empty dictionary to store clusters
clusters_dict = {}

# Process each line in the file
current_cluster = None
for line in lines:
    # Strip leading and trailing whitespaces
    line = line.strip()

    # Check if the line starts with 'Cluster'
    if line.startswith('Cluster'):
        current_cluster = line  # Set the current cluster name
        clusters_dict[current_cluster] = []  # Initialize an empty list for the cluster
    elif line.startswith('[') and line.endswith(']'):
        # If inside a cluster and the line is a list, convert it to a list and append to the dictionary
        clusters_dict[current_cluster] = line

# Convert string representations to actual lists using a custom function
def convert_to_list(s):
    try:
        # Assuming the string is enclosed in square brackets, remove extra whitespaces and evaluate as a list
        return [item.strip() for item in s[1:-1].split(',')]
    except (ValueError, TypeError, IndexError):
        # Handle errors gracefully
        return []

converted_clusters = {key: convert_to_list(value) for key, value in clusters_dict.items()}
sbert_clusters = {key: [value.strip("'") for value in values] for key, values in converted_clusters.items()}

#### Create df out of the cluster ####
dict_df={"cluster":sbert_clusters.keys(),"tags":sbert_clusters.values()}
tags_cluster=[]
for i in dict_df["tags"]:
    tags=[]
    for j in i:
        if " " in j:
            j=j.split()
            for k in j:
                tags.append(k)
        else:
            tags.append(j)
    tags_cluster.append(tags)
cluster_df=pd.DataFrame.from_dict(dict_df)

In [76]:
dict_df={"cluster":sbert_clusters.keys(),"tags":sbert_clusters.values()}
tags_cluster=[]
for i in dict_df["tags"]:
    tags=[]
    for j in i:
        if " " in j:
            j=j.split()
            for k in j:
                tags.append(k)
        else:
            tags.append(j)
    tags_cluster.append(tags)
cluster_df=pd.DataFrame.from_dict(dict_df)

In [77]:
cluster_df

,cluster,tags
0,28,"[clean energy, affordable energy, reduction of..."
1,43,"[stove, cooking, combustion, chamber, fed, lin..."
2,8,"[firewood, clay, insect, plant, agriculture, c..."
3,15,"[galvanized, metal, biofabricating, plastics, ..."
4,20,"[market, education, teachers, teacher, trainin..."
5,5,"[internal, inner, commuters, rural, global, ex..."
6,1,"[diameter, high, opening, open, scale, century..."
7,9,"[inlet, disguise, water supply, wind pump, pum..."
8,4,"[nature-based solutions, research, lab, constr..."
9,38,"[recycling, recycled, waste, waste management,..."


In [78]:
### Glove with normalisation ####
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nlp = spacy.load("en_core_web_md")
def central_word_cossim(row):
    # Calculate aggregated vector
    #vectors = [nlp(word).vector for word in words]
    
    ####Normalize first before mean, then cos_sim not required
    
    
    aggregated_vector = sum(nlp(word).vector/len(nlp(word).vector) for word in row) / len(row)
    #print(aggregated_vector)
    
    #Reshape to 2-D
    #word = word.reshape(1, -1)
    aggregated_vector = aggregated_vector.reshape(1, -1)
    
    
    # Calculate similarity between aggregated vector and each word vector
    similarities = [cosine_similarity(aggregated_vector, nlp(word).vector.reshape(1,-1)) for word in row]
    

    # Select the word with the highest similarity
    central_word_index = np.argmax(similarities)
    return row[central_word_index]
cluster_df["keywords_glove"]=cluster_df['tags'].apply(lambda row: central_word_cossim(row))
#cluster_df["keywords_glove_lemm"]=cluster_df['tags_lemmatized'].apply(lambda row: central_word_cossim(row))

In [79]:
cluster_df

,cluster,tags,keywords_glove
0,28,"[clean energy, affordable energy, reduction of...",sustainable energy
1,43,"[stove, cooking, combustion, chamber, fed, lin...",bread recycling
2,8,"[firewood, clay, insect, plant, agriculture, c...",deforestation
3,15,"[galvanized, metal, biofabricating, plastics, ...",granular
4,20,"[market, education, teachers, teacher, trainin...",unconventional partners
5,5,"[internal, inner, commuters, rural, global, ex...",neighborliness
6,1,"[diameter, high, opening, open, scale, century...",reconsidering
7,9,"[inlet, disguise, water supply, wind pump, pum...",clean water
8,4,"[nature-based solutions, research, lab, constr...",optimization
9,38,"[recycling, recycled, waste, waste management,...",waste separation


In [80]:
cluster_df.to_csv("level_1_root.csv")

# Level-2 clustering: Semantic clustering of the root word tags

In [81]:
combined_df = pd.read_csv("level_1_root.csv")

In [83]:
combined_df

,Unnamed: 0,cluster,tags,keywords_glove
0,0,28,"['clean energy', 'affordable energy', 'reducti...",sustainable energy
1,1,43,"['stove', 'cooking', 'combustion', 'chamber', ...",bread recycling
2,2,8,"['firewood', 'clay', 'insect', 'plant', 'agric...",deforestation
3,3,15,"['galvanized', 'metal', 'biofabricating', 'pla...",granular
4,4,20,"['market', 'education', 'teachers', 'teacher',...",unconventional partners
5,5,5,"['internal', 'inner', 'commuters', 'rural', 'g...",neighborliness
6,6,1,"['diameter', 'high', 'opening', 'open', 'scale...",reconsidering
7,7,9,"['inlet', 'disguise', 'water supply', 'wind pu...",clean water
8,8,4,"['nature-based solutions', 'research', 'lab', ...",optimization
9,9,38,"['recycling', 'recycled', 'waste', 'waste mana...",waste separation


In [82]:
combined_df['keywords_glove'] = combined_df['keywords_glove'].apply(ast.literal_eval)

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [ ]:
combined_df

In [84]:
combined_df_explode = combined_df.explode('keywords_glove')

In [85]:
combined_df_explode

,Unnamed: 0,cluster,tags,keywords_glove
0,0,28,"['clean energy', 'affordable energy', 'reducti...",sustainable energy
1,1,43,"['stove', 'cooking', 'combustion', 'chamber', ...",bread recycling
2,2,8,"['firewood', 'clay', 'insect', 'plant', 'agric...",deforestation
3,3,15,"['galvanized', 'metal', 'biofabricating', 'pla...",granular
4,4,20,"['market', 'education', 'teachers', 'teacher',...",unconventional partners
5,5,5,"['internal', 'inner', 'commuters', 'rural', 'g...",neighborliness
6,6,1,"['diameter', 'high', 'opening', 'open', 'scale...",reconsidering
7,7,9,"['inlet', 'disguise', 'water supply', 'wind pu...",clean water
8,8,4,"['nature-based solutions', 'research', 'lab', ...",optimization
9,9,38,"['recycling', 'recycled', 'waste', 'waste mana...",waste separation


In [86]:
print(combined_df_explode['keywords_glove'][0])

sustainable energy


In [89]:
combined_df_explode['unique_id'] = range(len(combined_df_explode))
combined_df_explode = combined_df_explode[['unique_id'] + [col for col in combined_df_explode.columns if col != 'unique_id']]
#print(combined_df_explode[combined_df_explode['keywords_glove'] == 'clean energy'].iloc[0])



In [90]:
combined_df_explode.set_index('unique_id', inplace=True)

In [91]:
combined_df_explode = combined_df_explode.applymap(lambda s: s.lower() if type(s) == str else s)

In [92]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [93]:
combined_corpus = combined_df_explode['keywords_glove'].unique()

In [94]:
combined_corpus = list(combined_corpus)

In [95]:
combined_corpus

['sustainable energy',
 'bread recycling',
 'deforestation',
 'granular',
 'unconventional partners',
 'neighborliness',
 'reconsidering',
 'clean water',
 'optimization',
 'waste separation',
 'digital forms sharing information',
 'actuators',
 'modular',
 'covid-19',
 'women participation',
 'electrical',
 'twinges',
 'month',
 'medically',
 'integrating',
 'enviromental mapping',
 'storing',
 'acculturation',
 'charts',
 'certificates',
 'value chain',
 'informal employment',
 'micro small medium enterprises',
 'wells',
 'weight',
 'digital tax submission',
 'prebiotics',
 'inequalities',
 'bar',
 'flaking',
 'espacio público',
 'iwartjqqnnaigquxcclhqjauhsjrcdokxzxlidvh_ckqzew_dplw',
 'page',
 'rational',
 'optometrists',
 'sdg localization',
 'simulation',
 'bootcamp',
 'biomarkers',
 'hydroxyapatite']

In [96]:
# corpus_embeddings = embedder.encode(corpus)
combined_corpus_embeddings = embedder.encode(combined_corpus, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [97]:
combined_corpus_embeddings

array([[-0.04145307,  0.14940245,  0.01436726, ..., -0.05649198,
         0.01895887,  0.05147467],
       [-0.05340147,  0.05057928, -0.00800286, ..., -0.10614591,
        -0.01811169, -0.07992048],
       [-0.01944282,  0.0164606 ,  0.06208973, ..., -0.12027527,
         0.01168747,  0.01831965],
       ...,
       [ 0.00300068, -0.07566634,  0.04738011, ..., -0.03460389,
        -0.06535251,  0.06300573],
       [-0.02887665, -0.00887529, -0.021031  , ..., -0.02115791,
         0.09023059, -0.01801626],
       [-0.07142466,  0.01307693,  0.07076944, ..., -0.01287552,
         0.12592988,  0.00792639]], dtype=float32)

In [98]:
combined_corpus_embeddings = combined_corpus_embeddings /  np.linalg.norm(combined_corpus_embeddings, axis=1, keepdims=True)

In [99]:
# clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
combined_clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold = 0.78, metric='cosine', linkage='average')
combined_clustering_model.fit(combined_corpus_embeddings)
combined_cluster_assignment = combined_clustering_model.labels_

In [100]:
combined_clustered_sentences = {}
for sentence_id, cluster_id in enumerate(combined_cluster_assignment):
    if cluster_id not in combined_clustered_sentences:
        combined_clustered_sentences[cluster_id] = []

    combined_clustered_sentences[cluster_id].append(combined_corpus[sentence_id])

In [101]:
combined_sorted_clusters = sorted(combined_clustered_sentences.items(), key=lambda x: x[0])
for i, (cluster_id, cluster) in enumerate(combined_sorted_clusters):
    print("Cluster: ", i+1)
    print(cluster)
    print("")

Cluster:  1
['optimization', 'value chain', 'simulation']

Cluster:  2
['medically', 'optometrists', 'biomarkers']

Cluster:  3
['unconventional partners', 'neighborliness', 'twinges']

Cluster:  4
['clean water', 'wells']

Cluster:  5
['month', 'bootcamp']

Cluster:  6
['granular', 'modular']

Cluster:  7
['charts', 'weight', 'bar', 'page']

Cluster:  8
['digital forms sharing information', 'certificates', 'digital tax submission']

Cluster:  9
['informal employment', 'micro small medium enterprises']

Cluster:  10
['sustainable energy', 'deforestation', 'enviromental mapping']

Cluster:  11
['acculturation', 'flaking']

Cluster:  12
['reconsidering', 'rational']

Cluster:  13
['espacio público']

Cluster:  14
['prebiotics']

Cluster:  15
['bread recycling', 'waste separation', 'storing']

Cluster:  16
['covid-19']

Cluster:  17
['sdg localization']

Cluster:  18
['iwartjqqnnaigquxcclhqjauhsjrcdokxzxlidvh_ckqzew_dplw']

Cluster:  19
['inequalities']

Cluster:  20
['integrating']

Clus

In [102]:
combined_cluster_assignment

array([ 9, 14,  9,  5,  2,  2, 11,  3,  0, 14,  7, 20,  5, 15, 22, 20,  2,
        4,  1, 19,  9, 14, 10,  6,  7,  0,  8,  8,  3,  6,  7, 13, 18,  6,
       10, 12, 17,  6, 11,  1, 16,  0,  4,  1, 21], dtype=int64)

In [103]:
combined_clustering_model.get_params()

{'affinity': 'deprecated',
 'compute_distances': False,
 'compute_full_tree': 'auto',
 'connectivity': None,
 'distance_threshold': 0.78,
 'linkage': 'average',
 'memory': None,
 'metric': 'cosine',
 'n_clusters': None}

In [104]:
type(combined_clustered_sentences)

dict

In [105]:
sbert_clusters = {key: [value.strip("'") for value in values] for key, values in combined_clustered_sentences.items()}

In [106]:
len(sbert_clusters.keys())

23

In [108]:
dict_df={"cluster":sbert_clusters.keys(),"tags":sbert_clusters.values()}
tags_cluster=[]
for i in dict_df["tags"]:
    tags=[]
    for j in i:
        if " " in j:
            j=j.split()
            for k in j:
                tags.append(k)
        else:
            tags.append(j)
    tags_cluster.append(tags)
cluster_df=pd.DataFrame.from_dict(dict_df)

In [109]:
#### Rake tags ####
from rake_nltk import Rake

# Function to generate a word similar in meaning using Rake
def generate_similar_word_rake(tag_list):
    # Join the tags to form a single text
    #nlp = spacy.load("en_core_web_md")
    text = ', '.join(tag_list)
    #print(tag_list)
    #print(text)
    
    #word_vector = np.mean([nlp(word).vector for word in tag_list if nlp(word).has_vector], axis=0)
    # Initialize Rake with a minimum word length of 1
    rake = Rake(min_length=1)
    #print(word_vector)
    # Extract keywords using Rake
    rake.extract_keywords_from_text(text)

    # Get the most important keyword
    keywords = rake.get_ranked_phrases_with_scores()
    
    # Return the most important keyword
    return keywords[0][1] if keywords else None
cluster_df["keywords_rake"]=cluster_df['tags'].apply(lambda row: generate_similar_word_rake(row))

In [112]:
cluster_df.to_csv("level_2_root.csv")

In [110]:
cluster_df

,cluster,tags,keywords_rake
0,9,"[sustainable energy, deforestation, enviroment...",sustainable energy
1,14,"[bread recycling, waste separation, storing]",waste separation
2,5,"[granular, modular]",modular
3,2,"[unconventional partners, neighborliness, twin...",unconventional partners
4,11,"[reconsidering, rational]",reconsidering
5,3,"[clean water, wells]",clean water
6,0,"[optimization, value chain, simulation]",value chain
7,7,"[digital forms sharing information, certificat...",digital forms sharing information
8,20,"[actuators, electrical]",electrical
9,15,[covid-19],covid


# Cooccurrence of tags in the clusters